## 현재 상황
- 편의점 감자튀김 (무게 130g으로 알려져있음)
- 학생이 무게 하나 측정, 122.02g으로 나옴
- 2주간 매우 감자튀김의 무게를 측정
  + 14개의 표본의 평균을 계산했는데 128.451g -> 편의점에 컴플레인
  + 편의점 입장은 그저 우연이다
- 14개 표본의 평균이 128.451g으로 나온 건 정말 우연일까?

### 질문
- 어떤 통계 검정을 사용해야 하는가?
- 귀무가설과 대립가설을 설정하라

In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
import scipy 
scipy.__version__

'1.11.4'

In [3]:
df=pd.read_csv('data/ch11_potato.csv')
sample=np.array(df['무게'])
sample

array([122.02, 131.73, 130.6 , 131.82, 132.05, 126.12, 124.43, 132.89,
       122.79, 129.95, 126.14, 134.45, 127.64, 125.68])

In [4]:
# 표본평균
s_mean=np.mean(sample)
s_mean

128.4507142857143

### A학생의 관심
- 표본평균이 모평균 130보다 작은지 여부
- 여기서 감자튀김의 표본집단이 따르고 있다 --> 정규성 검정을 할 필요가 있는가?
  + 정규성 검정없이 모수 검정을 하면 된다,
- 모분산이 9임을 알고 있다고 전제한다

In [5]:
rv=stats.norm(130,np.sqrt(9/14))
rv.isf(0.95) #유의수준 5%에서 128.681의 값은 귀무가설을 기각할 수 있다. 하지만 표본이 너무 적다

128.68118313069039

In [6]:
rv=stats.norm(130,np.sqrt(9/30))
rv.isf(0.95)

129.0990765647245

In [7]:
#모평균 130
z= (s_mean-130)/np.sqrt(9/14)
z

-1.932298779026813

In [8]:
#검정 통계량에 관한 임계값 구하기
rv=stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [9]:
z= (s_mean-130)/np.sqrt(9/30)
rv=stats.norm()
rv.isf(0.95)

-1.6448536269514722

In [10]:
rv.cdf(z) # p-value 귀무가설을 기각할 수 있다.

0.0023376354467074096

In [11]:
#양측검정
z=(s_mean-130)/np.sqrt(9/14)
rv=stats.norm()
rv.interval(0.95)

(-1.959963984540054, 1.959963984540054)

In [12]:
%precision 3
np.random.seed(1111)

In [13]:
rv.cdf(z)*2 #p-value 귀무가설 채택

0.053

## 가설검정의 오류 2가지
- 1종 오류: 귀무가설이 옳을 때 귀무가설을 기각하는 오류
- 2종 오류: 대립가설이 옳을 때 귀무가설을 채택하는 오류

### 제 1종 오류
- 실제로 평균이 130g인데도 불구하고 평균은 130보다 작다고 결론을 내리는 경우
- False Postive

In [14]:
rv=stats.norm(130,3)
c=stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z < c:
        cnt += 1
cnt / n_samples

0.053

- 1종 오류를 범할 비율은 0.053/ 약 5%의 비율로 130g보다 작다라고 잘못 탐지하는 것과 같다.
- 1종 오류를 범할 확률은 위험률
- 좀 더 엄격하게 적용하고 싶다고 하면 유의수준 1%에서 가설검정을 수행

## 제 2종 오류
- 실제로 표본평균이 130g보다 작음에도 불구하고 표본평균이 130g보다 작다라는 결론을 얻을 수 없는 상황
- 본래 검출해야 하는 것을 검출하지 못했으므로 미탐 false nagative

In [15]:
rv = stats.norm(128, 3)

In [16]:
rv = stats.norm(128, 3)
c = stats.norm().isf(0.95)
n_samples = 10000
cnt = 0
for _ in range(n_samples):
    sample_ = np.round(rv.rvs(14), 2)
    s_mean_ = np.mean(sample_)
    z = (s_mean_ - 130) / np.sqrt(9/14)
    if z >= c:
        cnt += 1
        
cnt / n_samples

0.197

- 0.199의 의미는 검정력 / 제대로 탐지하지 못할 확률이 약 20%
- 2종 오류는 언제나 모집단의 정보에 의존한다.
  + 중요한 포인트는 분석가는 모집단의 구체적인 정보를 알 방법이 없다.
  + 2종 오류는 컨트롤이 불가하다.

### 모분산을 안다고 가정했을 때
- 현실적으로 모집단을 아는 것은 불가능
  + 건강보험과 통계 / 전국민 데이터가 있는 자료
- 모분산을 알고 있다는 뜻은 모집단을 알고 있다 -> 모쳥균도 구할 수 있음 -> 모평균에 대한 검정을 굳이 할 필요가 있나?

In [17]:
def pmean_test(sample, mean0, alpha=0.05):
    s_mean = np.mean(sample)
    u_var = np.var(sample, ddof=1)
    n = len(sample)
    rv = stats.t(df=n-1)
    interval = rv.interval(1-alpha)

    t = (s_mean - mean0) / np.sqrt(u_var/n)
    print(f't통계량 값:{t:.3f}')
    if interval[0] <= t <= interval[1]:
        print('귀무가설을 채택')
    else:
        print('귀무가설을 기각')

    if t < 0:
        p = rv.cdf(t) * 2
    else:
        p = (1 - rv.cdf(t)) * 2
    print(f'p값은 {p:.3f}')

In [18]:
pmean_test(sample,130)

t통계량 값:-1.455
귀무가설을 채택
p값은 0.169


### 대응표본 t-검정
- A 학생은 친구20명에게 1주일간 근력 운동을 하게함
- 운동 전후에 집중력을 측정하는 테스트를 받게됨
- 가설검정
  + H0:운동 전과 후의 집중력의 평균차이는 없다
  + H1:운동 전과 후의 집중력의 평균차이는 있다

In [19]:
training_rel=pd.read_csv('data/ch11_training_rel.csv')
training_rel['차']=training_rel['전']-training_rel['후']

In [28]:
from scipy import stats
t, p=stats.ttest_1samp(training_rel['차'],0)
p #p-value

0.040

## 독랍검정 t-검정
- 학생이 늘음
  + A학생 그룹은 인문계열
  + B학생 그룹은 체육계열
- A그룹, B그룹 사이에 집중력 테스트의 평균에서 차이가 나지 않을까 생각하여 테스트함
- 이 데이터로부터 어떤 검정을 수행해야 A학생과 B학생 학급의 집중력에 유의미한 차이가 있는지 확인
- 가정설정
  + H0:그룹사이의 집중력의 평균차이는 없다  + 
H1:그룹사이의 집중력의 평균차이는 있
 
- 독립비교 t-검정 두가지 가정
  + 1) 각 그룹의 데이터의 정규성을 검정
  + 2) 두 그룹의 분산이 같다 (=등분산성 검정)다

In [30]:
training_ind=pd.read_csv('data/ch11_training_ind.csv')
training_ind.head()

,A,B
0,47,49
1,50,52
2,37,54
3,60,48
4,39,51


- H0:그룹사이의 집중력의 평균차이는 없다- 
H1:그룹사이의 집중력의 평균차이는 있다

In [33]:
t, p=stats.ttest_ind(training_ind['A'],training_ind['B'],equal_var=True)
t, p #p-value의 값이 0.05보다 크기때문에 귀무가설 기각

(-1.761, 0.086)

## 비모수 검정
- 모수 검정에서 각 가정을 만족시키지 못할 때, 비모수검정을 사용

## 윌콕슨의 부호순위검정
- 평균의 차이가 아님
- 중앙값 차이에 대한 검정

In [34]:
training_rel=pd.read_csv('data/ch11_training_rel.csv')
toy_df=training_rel[:6].copy()
toy_df

,전,후
0,59,41
1,52,63
2,55,68
3,61,59
4,59,84
5,45,37


In [35]:
# 순위 적용
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['순위'] = rank
toy_df

,전,후,순위
0,59,41,5
1,52,63,3
2,55,68,4
3,61,59,1
4,59,84,6
5,45,37,2


In [36]:
import numpy as np

r_minus = np.sum((diff < 0) * rank)
r_plus = np.sum((diff > 0) * rank)

r_minus, r_plus

(8, 13)

In [37]:
toy_df['후'] = toy_df['전'] + np.arange(1, 7)
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,순위,차
0,59,60,1,1
1,52,54,2,2
2,55,58,3,3
3,61,65,4,4
4,59,64,5,5
5,45,51,6,6


In [38]:
toy_df['후'] = toy_df['전'] + [1, -2, -3, 4, 5, -6]
diff = toy_df['후'] - toy_df['전']
rank = stats.rankdata(abs(diff)).astype(int)
toy_df['차'] = diff
toy_df['순위'] = rank
toy_df

,전,후,순위,차
0,59,60,1,1
1,52,50,2,-2
2,55,52,3,-3
3,61,65,4,4
4,59,64,5,5
5,45,39,6,-6
